#########################  LEIA-ME  ########################################

# Na mesma pasta do testeCorrespondencia.ipynb deve ter uma pasta chamada zipados e outrada chamada extraidos
# você deve colocar todos os arquivos a serem testados na pasta zipados
# no mesmo nível de diretório do arquivo testeCorrespondencia.ipynb você deve colocar o arquivo dadosElicao.xlsx

# código versão 1.0 
# autor: Anderson de Paiva Oliveira

############################################################################

In [10]:
import pandas as pd
import requests, bs4, os
from zipfile import ZipFile
import warnings

diretorioDoProjeto = "d://Users//marcio.melo//Documents//Script_Validacao_RDE//" 
pastaComZipados = diretorioDoProjeto + "zipados//"
pastaComExtraidos = diretorioDoProjeto + "extraidos//"

#warnings.simplefilter("ignore")
excel_file = pd.ExcelFile('dadosEleicao.xlsx')
df = excel_file.parse('Sheet1')
listaUfurnasSecao = df[["UF","Quantidade urna seção"]]
listaUfurnasContingencia = df[["UF","Quantidade urna contingência"]]

list = [] 
for (root, dirs, file) in os.walk(pastaComZipados): 
    for f in file: 
        with ZipFile(pastaComZipados + f, "r") as arqzip:
            arqzip.extractall("extraidos")
            
dir = os.listdir("extraidos")
caminhoextraidos = pastaComExtraidos
for file in dir:
    if file.startswith('cmrj') or file.startswith('leiame'):
        os.remove(caminhoextraidos+file)
    
dir = os.listdir("extraidos")
caminhoextraidos = pastaComExtraidos

quantidadeArquivosTestados = 0
quantidaDeFalhaSecao = 0
quantidaDeFalhaContingencia = 0
for file in dir:
    
    tabela = pd.read_csv(caminhoextraidos+file, encoding='latin-1', sep=";")
    totalDeRegistros = len(tabela)
    totalDeRegistros = '[' + str(totalDeRegistros) + ']'
    if file.startswith('ccont'):
        siglauf = file[9:11]
        turno = file[6:8]
        tipoarquivo = 'Contigência'
        quantidade = listaUfurnasContingencia.loc[(listaUfurnasContingencia['UF']==siglauf),['Quantidade urna contingência']]
        if not (totalDeRegistros==str(quantidade.iloc[0].values)):
            quantidaDeFalhaContingencia = quantidaDeFalhaContingencia + 1
       
    elif file.startswith('csec'):
        siglauf = file[8:10]
        turno = file[5:7]
        tipoarquivo = 'Seção'
        quantidade = listaUfurnasSecao.loc[(listaUfurnasSecao['UF']==siglauf),['Quantidade urna seção']]
        if not (totalDeRegistros==str(quantidade.iloc[0].values)):
            quantidaDeFalhaSecao = quantidaDeFalhaSecao + 1
    
    quantidadeArquivosTestados = quantidadeArquivosTestados + 1
    
    print("O tipo do arquivo é: "+ tipoarquivo +", a UF do arquivo é: "+str(siglauf) + ", o turno é: "+ turno +", o arquivo tem " + str(totalDeRegistros) + " linhas e deveria ter: " + str(quantidade.iloc[0].values))
    print("o resultado da comparação para a UF: " +str(siglauf)+" é: ", end=""); print (totalDeRegistros==str(quantidade.iloc[0].values))
    print("\n")
print("Quantidade de arquivos de seção inconsistentes: " + str(quantidaDeFalhaSecao))
print("Quantidade de arquivos de contingência inconsistentes: " + str(quantidaDeFalhaContingencia))
print("Total de inconsistências: " + str(quantidaDeFalhaSecao+quantidaDeFalhaContingencia))
print("Quantidade de arquivos testados: " + str(quantidadeArquivosTestados))

FileNotFoundError: [WinError 2] O sistema não pode encontrar o arquivo especificado: 'd://Users//marcio.melo//Documents//Script_Validação_RDE//extraidos//cmrj_1t_DF_011020221232.csv'